In [3]:
import xarray as xr

# Example: ERA5 monthly mean u-wind and temperature at pressure levels
ds = xr.open_dataset('/Users/ryaneagan/Downloads/met_test_data_500hpa.nc')

u = ds['u']  # [time, level, lat, lon]
T = ds['t']  # Temperature
lat = ds['latitude']
#p = ds['level'] * 100  # hPa to Pa
p = 500

In [4]:
u_zm = u.mean(dim='longitude')
T_zm = T.mean(dim='longitude')

Calc background PV gradient $ q_{y} $ for QG in pressure coordinates

$$ q_{y} \approx \beta - \frac{\partial^2\bar{u}}{a^2 \cos^2(\phi) \partial\phi^2} + (vertical shear term) $$

This just computes the barotropic contribution

In [6]:
import numpy as np

Re = 6.371e6  # Earth radius in meters
omega = 7.2921e-5  # Earth's rotation rate
beta = (2 * omega / Re) * np.cos(np.radians(lat))

# d²u/dφ² in spherical coordinates (approximate finite difference)
phi = np.radians(lat)
cosphi = np.cos(phi)

dphi = np.gradient(phi)
d2u_dphi2 = np.gradient(np.gradient(u_zm.values, dphi, axis=-1), dphi, axis=-1)

q_y = beta - d2u_dphi2 / (Re**2 * cosphi**2)

ValueError: applied function returned data with an unexpected number of dimensions. Received 3 dimension(s) but expected 1 dimensions with names ('latitude',), from:

array([[[ 2.851191e+17,           nan, ...,           nan,           nan]],

       [[ 1.001013e+17,           nan, ...,           nan,           nan]],

       ...,

       [[-1.974042e+16,           nan, ...,           nan,           nan]],

       [[-7.002551e+16,           nan, ...,           nan,           nan]]],
      shape=(72, 1, 121))

Computer the refractive index $ n^2 $

$$ n^2 = \frac{q_{y}}{\bar(u)}-\frac{s^2}{a^2\cos^2\phi} $$

In [ ]:
s = 1  # Zonal wavenumber
cos2phi = np.cos(phi)**2

n_squared = (q_y / u_zm) - (s**2 / (Re**2 * cos2phi))

Plot it. Regions of $ n^2 > 0 $ : Rossby waves of wavenumber s can propagate
Contours where $ n^2 = 0 $ : Turing surfaces, act as natural boundaries for waveguides (often aligned with jet streams)

In [ ]:
import matplotlib.pyplot as plt

# Choose a specific time slice or average over time
n2 = n_squared.sel(time='2020-01-01')  # or n_squared.mean(dim='time')

plt.contourf(lat, p/100, n2.T, levels=np.linspace(-1e-10, 1e-10, 21), cmap='coolwarm', extend='both')
plt.contour(lat, p/100, n2.T, levels=[0], colors='black', linewidths=1.5)  # waveguide boundary
plt.gca().invert_yaxis()
plt.xlabel('Latitude')
plt.ylabel('Pressure (hPa)')
plt.title(f'Refractive Index Squared $n^2$ (s={s}) – ERA5')
plt.colorbar(label='$n^2$ (1/m²)')
plt.grid(True)
plt.show()

Full script to load data and compute all terms.

The script assumes you have a NetCDF file (e.g., ERA5_zonal_wind_temp.nc) with at least:
	•	u(time, level, lat, lon) – Zonal wind
	•	t(time, level, lat, lon) – Temperature

You can obtain this from Copernicus Climate Data Store or another provider like NOAA.

	•	Positive n^2 → wave can propagate
	•	Zero n^2 → turning surfaces / waveguide boundaries
	•	Negative n^2 → evanescent zone (wave reflection or decay)

Regions with positive n^2 bounded by zero contours are waveguides — common near jet streams or in the stratosphere.

In [29]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from metpy.calc import potential_temperature, first_derivative
from metpy.units import units

# ----------------------------------------
# 1. Load ERA5 Data and Attach Units
# ----------------------------------------
ds = xr.open_dataset('/Users/ryaneagan/Downloads/refractive_test.nc')
ds = ds.metpy.parse_cf()  # Attach units from CF metadata

# Use correct dimension name: 'pressure_level'
u = ds['u']  # Zonal wind (m/s)
T = ds['t']  # Temperature (K)
lat = ds['latitude']
plev = ds['pressure_level']  # Pressure levels (in hPa)

# ----------------------------------------
# 2. Compute Zonal Means
# ----------------------------------------
u_zm = u.mean(dim='longitude')  # shape: [time, level, lat]
T_zm = T.mean(dim='longitude')

# ----------------------------------------
# 3. Prepare Pressure as xarray DataArray with Units
# ----------------------------------------
p_levels = plev.values * 100.0 * units.pascal  # convert hPa → Pa

# Create a 1D DataArray with units
p_da = xr.DataArray(p_levels, coords={"pressure_level": plev}, dims=["pressure_level"])

# Expand to match [time, pressure_level, latitude]
p_expanded = p_da.expand_dims({
    "time": T_zm.time,
    "latitude": T_zm.latitude
}).transpose("time", "pressure_level", "latitude")

# ----------------------------------------
# 4. Compute Potential Temperature
# ----------------------------------------
theta = potential_temperature(p_expanded, T_zm)

# ----------------------------------------
# 5. Compute Vertical Derivatives
# ----------------------------------------
dtheta_dp = first_derivative(theta, p_levels, axis=1)
du_dp = first_derivative(u_zm, p_levels, axis=1)

# ----------------------------------------
# 6. Compute Meridional PV Gradient q_y
# ----------------------------------------
Re = 6.371e6  # Earth radius [m]
omega = 7.2921e-5  # Earth's rotation rate [rad/s]
phi = np.radians(lat)
cosphi = np.cos(phi)
cos2phi = cosphi ** 2
beta = (2 * omega / Re) * cosphi

# Compute ∂²u/∂φ² (second meridional derivative)
dphi = np.gradient(phi)
d2u_dphi2 = np.gradient(np.gradient(u_zm, dphi, axis=-1), dphi, axis=-1)

# First part of q_y
q_y = beta - d2u_dphi2 / (Re ** 2 * cos2phi)

# Compute f²/∂θ * ∂u/∂p vertical term
f = 2 * omega * np.sin(phi)
f2 = f ** 2
strat_term = np.zeros_like(u_zm)

for i, lat_i in enumerate(lat):
    f2_over_dtheta = f2[i] / dtheta_dp[:, :, i]
    term = np.gradient(f2_over_dtheta * du_dp[:, :, i], p_levels * units.pascal, axis=0)
    strat_term[:, :, i] = term

q_y += strat_term

# ----------------------------------------
# 7. Compute Refractive Index Squared n²
# ----------------------------------------
s = 1  # zonal wavenumber
n_squared = (q_y / u_zm) - (s ** 2 / (Re ** 2 * cos2phi))

# Mask regions where u ≈ 0 to avoid division issues
n_squared = n_squared.where(np.abs(u_zm) > 1.0)

# ----------------------------------------
# 8. Plot n² (Latitude–Pressure Diagram)
# ----------------------------------------
n2_plot = n_squared.sel(time=n_squared.time[0])  # Choose first time or average

plt.figure(figsize=(10, 6))
levels = np.linspace(-1e-10, 1e-10, 21)

cf = plt.contourf(
    lat,
    plev,
    n2_plot.T,
    levels=levels,
    cmap='RdBu_r',
    extend='both'
)

# Plot n² = 0 line — waveguide boundary
plt.contour(
    lat,
    plev,
    n2_plot.T,
    levels=[0],
    colors='black',
    linewidths=1.2
)

plt.gca().invert_yaxis()
plt.title(r"Refractive Index Squared $n^2$ for Stationary Rossby Waves (s=1)")
plt.xlabel("Latitude (°)")
plt.ylabel("Pressure (hPa)")
plt.colorbar(cf, label=r"$n^2$ (m$^{-2}$)")
plt.grid(True)
plt.tight_layout()
plt.show()

AttributeError: 'DataArray' object has no attribute 'time'